<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;"><a href="http://www.editorialuoc.cat/mineria-de-datos" target="_blank">Minería de datos: Modelos y algoritmos</a></p>
<p style="margin: 0; text-align:right;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Universitat Oberta de Catalunya (<b>UOC</b>)</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# CH12EJ1 Redes neuronales

## 1. Carga de datos y tareas de preprocesado

Trabajaremos con el paquete R `neuralnet` y con el juego de datos [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+%28diagnostic%29).

Veremos las distintas posibilidades de una read neuronal: capas, neuronas y criterio de parada.

### Instalación y carga de paquetes R

Para terminar, se cargan todos los paquetes que van a ser utilizados a lo largo del ejercicio.

In [13]:
if(!require(neuralnet)){
    install.packages('neuralnet', repos='http://cran.es.r-project.org')
    require(neuralnet)
}

### Manipulación y preparación de los datos

In [14]:
dataRaw <- read.table(  url("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wpbc.data"), 
                  sep=",", header=FALSE)

# Creamos una nueva variable T o target donde almacenamos el objetivo a predecir
# Como la función R neuralnet() solo admite valores numéricos, convertiremos N en 0 y R en 1
dataRaw$T <- as.character(dataRaw$V2)
dataRaw$T[dataRaw$T == "N"] <- 0
dataRaw$T[dataRaw$T == "R"] <- 1
dataRaw$T <- as.numeric(dataRaw$T)

# Visualizar un resumen de los datos
summary(dataRaw)
head(dataRaw)

       V1          V2            V3               V4              V5       
 Min.   :   8423   N:151   Min.   :  1.00   Min.   :10.95   Min.   :10.38  
 1st Qu.: 855745   R: 47   1st Qu.: 14.00   1st Qu.:15.05   1st Qu.:19.41  
 Median : 886339           Median : 39.50   Median :17.29   Median :21.75  
 Mean   :1990469           Mean   : 46.73   Mean   :17.41   Mean   :22.28  
 3rd Qu.: 927996           3rd Qu.: 72.75   3rd Qu.:19.58   3rd Qu.:24.66  
 Max.   :9411300           Max.   :125.00   Max.   :27.22   Max.   :39.28  
                                                                           
       V6               V7               V8                V9         
 Min.   : 71.90   Min.   : 361.6   Min.   :0.07497   Min.   :0.04605  
 1st Qu.: 98.16   1st Qu.: 702.5   1st Qu.:0.09390   1st Qu.:0.11020  
 Median :113.70   Median : 929.1   Median :0.10190   Median :0.13175  
 Mean   :114.86   Mean   : 970.0   Mean   :0.10268   Mean   :0.14265  
 3rd Qu.:129.65   3rd Qu.:1193.5   3r

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V27,V28,V29,V30,V31,V32,V33,V34,V35,T
,<int>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,⋯,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5,0
2,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,⋯,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2,0
3,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,⋯,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0,0
4,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,⋯,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0,0
5,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,⋯,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0,1
6,843786,R,77,12.75,15.29,84.60,502.7,0.11890,0.1569,0.1664,⋯,733.2,0.1706,0.4196,0.5999,0.1709,0.3485,0.11790,2.5,0,1


### Datos de entrenamiento y datos de prueba

In [15]:
# Transferimos la variable objetivo a la primera posición de la tabla
dataRaw[,3] <- as.numeric(dataRaw[,36])


# Separamos los datos de las etiquetas
labels <- dataRaw[,2]
data <- dataRaw[,3:35]

# Seleccionamos el 90% para entrenamiento y el 10% para test
NObs <-  nrow(data)
NTrain <- round(NObs*0.9)
NTest <- NObs - NTrain

# Definir el conjunto de entrenamiento:
train <- data[1:NTrain,]
labelsTrain <- labels[1:NTrain]

# Definir el conjunto de test:
test <- data[(NTrain+1):NObs,]
labelsTest <- labels[(NTrain+1):NObs]

head(train)

,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,⋯,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
2,0,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,⋯,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
3,0,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,⋯,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
4,0,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,⋯,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
5,1,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,⋯,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0
6,1,12.75,15.29,84.60,502.7,0.11890,0.1569,0.1664,0.07666,0.1995,⋯,107.30,733.2,0.1706,0.4196,0.5999,0.1709,0.3485,0.11790,2.5,0


## 2. Creación de los modelos

### Modelo con una capa de 2 neuronas y con un criterio de parada de 0.01

In [16]:
# --------------------------------
# Modelo con una capa de 2 neuronas y con un criterio de parada de 0.01
# --------------------------------
set.seed(10)
net.1 <- neuralnet(V3~V4+V5,data=train, 
                        hidden=2, threshold=0.01)

print(net.1)

$call
neuralnet(formula = V3 ~ V4 + V5, data = train, hidden = 2, threshold = 0.01)

$response
    V3
1    0
2    0
3    0
4    0
5    1
6    1
7    0
8    1
9    0
10   0
11   0
12   0
13   0
14   1
15   0
16   1
17   0
18   1
19   0
20   0
21   0
22   0
23   0
24   0
25   0
26   0
27   0
28   0
29   0
30   0
31   0
32   0
33   0
34   0
35   1
36   0
37   0
38   1
39   0
40   1
41   1
42   0
43   1
44   1
45   0
46   0
47   1
48   0
49   1
50   0
51   1
52   0
53   0
54   0
55   0
56   0
57   1
58   0
59   0
60   1
61   0
62   0
63   1
64   0
65   0
66   0
67   0
68   1
69   0
70   1
71   0
72   0
73   0
74   0
75   0
76   1
77   0
78   0
79   0
80   0
81   0
82   1
83   0
84   0
85   0
86   0
87   0
88   0
89   1
90   0
91   0
92   0
93   0
94   0
95   0
96   0
97   0
98   0
99   0
100  0
101  0
102  0
103  0
104  0
105  0
106  1
107  0
108  0
109  0
110  1
111  1
112  0
113  1
114  0
115  1
116  0
117  0
118  0
119  0
120  0
121  1
122  0
123  0
124  0
125  0
126  1
127  0
128  0
12

In [17]:
# Obtenemos un resumen del nivel de precisión del modelo
net.1$result.matrix[1:3,]

error reached.threshold             steps 
     1.607575e+01      9.760344e-03      1.751600e+04

In [18]:
# Predicción
net.predict1 <- compute(net.1, test[,2:3]) 
pred1 <- round(net.predict1$net.result)

# Generamos una matriz de confusión para medir el
# grado de acierto de la predicción
t1 <- table(pred = pred1, real = test[,1])
print(t1)
print(paste("Accuracy =", round(100 * sum(diag(t1)) / sum(t1), digits=2)))

    real
pred  0  1
   0 19  1
[1] "Accuracy = 95"


### Modelo de 3 capas de 3, 2 y 1 neuronas y con un criterio de parada de 0.01

In [19]:
# --------------------------------
# Modelo con 3 capas de 3, 2 y 1 neuronas respectivamente y con un criterio de parada de 0.01
# --------------------------------
set.seed(10)
net.2 <- neuralnet(V3~V4+V5,data=train, 
                        hidden=c(3,2,1), threshold=0.01)


In [20]:
# Obtenemos un resumen del nivel de precisión del modelo
net.2$result.matrix[1:3,]

error reached.threshold             steps 
     1.510758e+01      8.317010e-03      3.170000e+03

In [21]:
# Predicción
net.predict2 <- compute(net.2, test[,2:3]) 
pred2 <- round(net.predict2$net.result)

# Generamos una matriz de confusión para medir el
# grado de acierto de la predicción
t2 <- table(pred = pred2, real = test[,1])
print(t2)
print(paste("Accuracy =", round(100 * sum(diag(t2)) / sum(t2), digits=2)))

    real
pred  0  1
   0 16  0
   1  3  1
[1] "Accuracy = 85"
